BI名企班 谢雅楠

### Thinking

1. 线性规划与混合整数规划的区别是什么?  
线性规划（LP），目标函数是线性的，所有约束也是线性的，决策变量可以取任何的实数（只对变量X限定了范围，但没有限定类型）  
混合整数规划（MILP），优化问题里不止有continous variable,也有integer variable  

2. 针对VRP问题，常见的约束条件都有哪些?  
车辆具有可携带的最大重量或数量  
司机需要在指定时间窗口内访问某位置  
点的访问顺序等  

### Action1 Santa的接待安排

Santa的接待安排  
圣诞节前100天，Santa开放了workshop，欢迎以家庭单位的参观，如何更合理的安排这些家庭参观？  
每个家庭有10个选择choice0-9，数字代表了距离圣诞节的天数，比如 1代表12月24日，每个家庭必须并且只安排一次参观  
家庭数量 5000，即family_id 为[0, 4999]，每天访问的人数需要在125-300人
为了更合理的计算Santa的安排能力，我们使用preference cost和accounting penalty两个指标  
1）preference cost，代表Santa的个性化安排能力  
2）accounting penalty，代表Santa安排的财务成本  
每天接待的人员数N(d)如果大于125，就会拥挤，产生过多的清洁成本  
最终的 Score = preference cost + accounting penalty  
最终提交每个家庭的安排 submission.csv


In [3]:
import pandas as pd
import numpy as np

#### Step1, 数据加载

In [4]:
data = pd.read_csv("family_data.csv", index_col="family_id")
data

,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people
family_id,,,,,,,,,,,
0,52,38,12,82,33,75,64,76,10,28,4
1,26,4,82,5,11,47,38,6,66,61,4
2,100,54,25,12,27,82,10,89,80,33,3
3,2,95,1,96,32,6,40,31,9,59,2
4,53,1,47,93,26,3,46,16,42,39,4
...,...,...,...,...,...,...,...,...,...,...,...
4995,16,1,66,33,18,70,56,46,86,60,4
4996,88,66,20,17,26,54,81,91,59,48,2
4997,32,66,54,17,27,21,74,81,3,7,6


#### Step2，数据预处理
1）计算Perference Cost矩阵 pcost_mat  
2）计算Accounting Cost矩阵 acost_mat  
3）计算每个家庭的人数 FAMILY_SIZE  
4）每个家庭的倾向选择（choice_） DESIRED


In [5]:
## 1. preference cost

# n 家庭成员个数，如果满足choice的需求，需要的penalty
def get_penalty(n, choice):
    penalty = None
    if choice == 0:
        penalty = 0
    if choice == 1:
        penalty = 50
    if choice == 2:
        penalty = 50+9*n
    if choice == 3:
        penalty = 100+9*n
    if choice == 4:
        penalty = 200+9*n
    if choice == 5: 
        penalty = 100+18*n
    if choice == 6:
        penalty = 300+18*n
    if choice == 7: 
        penalty = 300+36*n
    if choice == 8: 
        penalty = 400+36*n
    if choice == 9:
        penalty = 500+(36+199)*n
    if choice > 9:
        penalty = 500+(36+398)*n
    return penalty

In [6]:
N_DAYS = 100
N_FAMILY = 5000
MIN_OCCUPANCY = 125
MAX_OCCUPANCY = 300

## pcost
# 每个家庭在什么时候（day 0-99）访问的penalty
pcost_mat = np.full(shape=(N_FAMILY, 100), fill_value=99999)
for f in range(N_FAMILY):
    f_num = data.loc[f, 'n_people']
#     print(f_num)
    # 第f个家庭，初始化pcost=choice11
    pcost_mat[f,:] = get_penalty(f_num, 10)
    for choice in range(10):
        temp = data.loc[f][choice]
        penalty = get_penalty(f_num, choice)
        pcost_mat[f, temp-1] = penalty

pcost_mat

array([[2236, 2236, 2236, ..., 2236, 2236, 2236],
       [2236, 2236, 2236, ..., 2236, 2236, 2236],
       [1802, 1802, 1802, ..., 1802, 1802,    0],
       ...,
       [3104, 3104,  616, ..., 3104, 3104, 3104],
       [ 390, 2670, 2670, ..., 2670, 2670, 2670],
       [2236, 2236, 2236, ..., 2236, 2236, 2236]])

In [7]:
## acost_mat
# 前一天的参观人数，当天的参观人数
acost_mat = np.zeros(shape=(MAX_OCCUPANCY+1,MAX_OCCUPANCY+1), dtype=np.float64)
for i in range(acost_mat.shape[0]):
    for j in range(acost_mat.shape[1]):
        diff = abs(i-j)
        acost_mat[i,j] = max(0,(i-125) /400 *i **(0.5+diff/50.0))
        
acost_mat

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [4.16316072e+15, 3.71482922e+15, 3.31477861e+15, ...,
        7.46610759e+00, 8.36716954e+00, 9.37697794e+00],
       [4.79555148e+15, 4.27883100e+15, 3.81778713e+15, ...,
        8.43020770e+00, 7.52185316e+00, 8.43020770e+00],
       [5.52415954e+15, 4.92860244e+15, 4.39725208e+15, ...,
        9.51970597e+00, 8.49339085e+00, 7.57772228e+00]])

In [8]:
## FAMILY_SIZE
FAMILY_SIZE=data['n_people'].values
FAMILY_SIZE

array([4, 4, 3, ..., 6, 5, 4])

In [9]:
## DESIRED
DESIRED = data.values[:,:-1] -1
DESIRED

array([[51, 37, 11, ..., 75,  9, 27],
       [25,  3, 81, ...,  5, 65, 60],
       [99, 53, 24, ..., 88, 79, 32],
       ...,
       [31, 65, 53, ..., 80,  2,  6],
       [66, 91,  3, ..., 11, 25, 69],
       [12, 10, 24, ..., 38, 17, 46]])

#### Step3，使用LP和MIP求解 规划方案
1）先使用LP 对绝大部分家庭进行规划  
2）再使用MIP 对剩余家庭进行规划  
3）汇总两边的结果 => 最终规划方案  

In [10]:
from ortools.linear_solver import pywraplp

In [11]:
## lp 
def solveLP():
    solver= pywraplp.Solver('AssignmentProblem', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    x = {}
    candidates = [[] for x in range(N_DAYS)]
    
    for i in range(N_FAMILY):
        for j in DESIRED[i, :]:
            candidates[j].append(i)
            # 决策变量
            x[i,j] = solver.BoolVar('x[%i, %i]' %(i,j))
            
    daily_occupancy = [solver.Sum([x[i,j] * FAMILY_SIZE[i] for i in candidates[j]]) for j in range(N_DAYS)]
    
    
    family_presence = [solver.Sum(x[i,j] for j in DESIRED[i,:]) for i in range(N_FAMILY)]
    
    # 目标函数
    preference_cost = solver.Sum([pcost_mat[i,j] * x[i,j] for i in range(N_FAMILY) for j in  DESIRED[i,:]])

    solver.Minimize(preference_cost)
    
    #认为约束
    for j in range(N_DAYS-1):
        solver.Add(daily_occupancy[j] - daily_occupancy[j+1] <= 25)
        solver.Add(daily_occupancy[j+1] - daily_occupancy[j] <= 25)
    
    for i in range(N_FAMILY):
        solver.Add(family_presence[i] == 1)
    
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j] >= MIN_OCCUPANCY)
        solver.Add(daily_occupancy[j] <= MAX_OCCUPANCY)
        
    result = solver.Solve()
    
    
    ## result
    temp = [(i,j, x[i,j].solution_value()) for i in range(N_FAMILY) for j in DESIRED[i,:] if x[i,j].solution_value() >0]
    
    df = pd.DataFrame(temp, columns=['family_id', 'day', 'result'])
    
    return df



In [12]:
%%time
result = solveLP()
result

CPU times: user 10.9 s, sys: 97.4 ms, total: 11 s
Wall time: 11 s


,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5067,4995,15,1.0
5068,4996,87,1.0
5069,4997,31,1.0
5070,4998,91,1.0


In [13]:
THRS = 0.999
assigned_df = result[result.result > THRS]
assigned_df

,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5067,4995,15,1.0
5068,4996,87,1.0
5069,4997,31,1.0
5070,4998,91,1.0


In [14]:
# 没有安排的家庭
unassigned_df = result[(result.result < THRS) & (result.result > 1-THRS)] 
unassigned_df

,family_id,day,result
16,16,45,0.750000
17,16,49,0.250000
60,59,38,0.875000
61,59,14,0.125000
190,188,9,0.666667
...,...,...,...
4956,4886,98,0.750000
4984,4914,38,0.500000
4985,4914,43,0.500000
5032,4961,53,0.750000


In [15]:
### 对剩余家庭进行规划
assigned_df['family_size'] = FAMILY_SIZE[assigned_df.family_id]
assigned_df

<ipython-input-15-9bdd94f460fb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assigned_df['family_size'] = FAMILY_SIZE[assigned_df.family_id]


,family_id,day,result,family_size
0,0,51,1.0,4
1,1,25,1.0,4
2,2,99,1.0,3
3,3,1,1.0,2
4,4,52,1.0,4
...,...,...,...,...
5067,4995,15,1.0,4
5068,4996,87,1.0,2
5069,4997,31,1.0,6
5070,4998,91,1.0,5


In [16]:
occupancy = assigned_df.groupby('day').family_size.sum().values
occupancy

array([275, 271, 299, 299, 270, 242, 223, 246, 269, 298, 290, 300, 275,
       250, 238, 272, 298, 298, 274, 248, 223, 241, 259, 286, 298, 298,
       273, 249, 239, 255, 278, 281, 252, 228, 209, 184, 202, 233, 253,
       231, 209, 183, 204, 225, 247, 278, 251, 227, 204, 198, 223, 248,
       260, 228, 208, 185, 173, 196, 220, 198, 162, 148, 119, 114, 149,
       175, 189, 160, 136, 119, 125, 135, 158, 185, 158, 127, 125, 125,
       142, 167, 185, 167, 141, 120, 128, 155, 174, 200, 179, 152, 130,
       122, 136, 158, 181, 157, 134, 125, 122, 124])

In [17]:
min_occupancy = np.array([max(0, MIN_OCCUPANCY - x) for x in occupancy])
min_occupancy

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6, 11,  0,  0,  0,  0,
        0,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5,  0,
        0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  3,  1])

In [18]:
max_occupancy = np.array([ MAX_OCCUPANCY - x for x in occupancy])
max_occupancy

array([ 25,  29,   1,   1,  30,  58,  77,  54,  31,   2,  10,   0,  25,
        50,  62,  28,   2,   2,  26,  52,  77,  59,  41,  14,   2,   2,
        27,  51,  61,  45,  22,  19,  48,  72,  91, 116,  98,  67,  47,
        69,  91, 117,  96,  75,  53,  22,  49,  73,  96, 102,  77,  52,
        40,  72,  92, 115, 127, 104,  80, 102, 138, 152, 181, 186, 151,
       125, 111, 140, 164, 181, 175, 165, 142, 115, 142, 173, 175, 175,
       158, 133, 115, 133, 159, 180, 172, 145, 126, 100, 121, 148, 170,
       178, 164, 142, 119, 143, 166, 175, 178, 176])

In [19]:
def solveIP(families, min_occupancy, max_occupancy):
    solver = pywraplp.Solver('AssignmentProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    
    n_families = len(families)
    
    x = {}
    candidates = [[] for x in range(N_DAYS)]
    
    for i in families:
        for j in DESIRED[i, :]:
            candidates[j].append(i)
            # 决策变量
            x[i,j] = solver.BoolVar('x[%i, %i]' %(i,j))
            
    daily_occupancy = [solver.Sum([x[i,j] * FAMILY_SIZE[i] for i in candidates[j]]) for j in range(N_DAYS)]
    
    
    family_presence = [solver.Sum(x[i,j] for j in DESIRED[i,:]) for i in families]
    
    # 目标函数
    preference_cost = solver.Sum([pcost_mat[i,j] * x[i,j] for i in families for j in  DESIRED[i,:]])

    solver.Minimize(preference_cost)
    
    #认为约束
    for j in range(N_DAYS-1):
        solver.Add(daily_occupancy[j] - daily_occupancy[j+1] <= 25)
        solver.Add(daily_occupancy[j+1] - daily_occupancy[j] <= 25)
    
    for i in range(n_families):
        solver.Add(family_presence[i] == 1)
    
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j] >= min_occupancy[j])
        solver.Add(daily_occupancy[j] <= max_occupancy[j])
        
    result = solver.Solve()
    
    ## result
    temp = [(i,j) for i in families for j in DESIRED[i,:] if x[i,j].solution_value() >0]
    
    df = pd.DataFrame(temp, columns=['family_id', 'day'])
    
    return df

In [20]:
unassigned = unassigned_df.family_id.unique()

result1 = solveIP(unassigned, min_occupancy, max_occupancy)

In [21]:
result1

,family_id,day
0,16,45
1,59,38
2,188,19
3,240,32
4,455,0
...,...,...
59,4619,67
60,4870,69
61,4886,98
62,4914,38


In [22]:
## 汇总结果
df = pd.concat((assigned_df[['family_id', 'day']], result1)).sort_values('family_id')
df

,family_id,day
0,0,51
1,1,25
2,2,99
3,3,1
4,4,52
...,...,...
5067,4995,15
5068,4996,87
5069,4997,31
5070,4998,91


#### Step4, 结果评估
按照evaluation标准，计算
Score = preference cost + accounting penalty


In [23]:
from numba import njit

# @njit(fastmath=True)
def pcost(prediction):
    daily_occupancy = np.zeros(N_DAYS+1, dtype=np.int64)
    penalty = 0
    for (i,p) in enumerate(prediction):
        n = FAMILY_SIZE[i]
        penalty += pcost_mat[i,p]
        daily_occupancy[p] += n
    return penalty, daily_occupancy


# @njit(fastmath=True)
def acost(daily_occupancy):
    accounting_cost = 0
    number_outofrange = 0
    daily_occupancy[-1] = daily_occupancy[-2]
    
    for day in range(N_DAYS):
        n_p1 = daily_occupancy[day+1]
        n = daily_occupancy[day]
        
        number_outofrange += (n>MAX_OCCUPANCY) or (n<MIN_OCCUPANCY)
        accounting_cost += acost_mat[n, n_p1]
    return accounting_cost, number_outofrange


# @njit(fastmath=True)
def cost_function(prediction):
    penalty, daily_occupancy = pcost(prediction)
    accounting_cost, number_outofrange = acost(daily_occupancy)
    final_score = penalty + accounting_cost + number_outofrange*99999999
    return final_score

In [24]:
prediction = df.day.values
print(cost_function(prediction))

77305.71527468342


In [25]:
def save_result(pred, filename):
    result = pd.DataFrame(range(N_FAMILY), columns=['family_id'])
    result['assigned_day'] = pred+1
    result.to_csv(filename, index=False)
    return result
result = save_result(prediction, 'submission.csv')

#### Step5，方案优化
通过更换family_id的选择，查找更好的score
每次更换后，都对方案进行评估，选择更小的score方案


In [26]:
def find_better(pred):
    fobs = np.argsort(FAMILY_SIZE)
#     print(fobs)
    score = cost_function(pred)
    original_score = np.inf
    
    while score<original_score:
        original_score = score
        for family_id in fobs:
            for pick in range(10):
                day = DESIRED[family_id, pick]
                # 该family的原有日期oldvalue
                oldvalue = pred[family_id]
                pred[family_id] = day
                new_score = cost_function(pred)
                if new_score < score:
                    score = new_score
                else:
                    pred[family_id] = oldvalue
    return score
    

In [29]:
new = prediction.copy()
find_better(new)
print(int(find_better(new)))

74280


In [30]:
result = save_result(new, 'submission2.csv')

### Action2 多辆车的路径规划VRP

多辆车的路径规划 VRP：  
条件：经过中国33个城市，一共4辆车，每辆车最大行驶10000公里  
目标：使得每辆车的行驶里程数更接近  
需要注意：  
在VRP问题中，路径上给点赋的index和点实际的index不一样，需要使用IndexToNode方法进行转换才能得到实际的index


In [13]:
import pandas as pd

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [38]:
class tsp(object):
    def __init__(self, city_names=None, num_vehicles=1):
        self.df = pd.read_excel('cities.xlsx')
        self.all_city = self.df['name'].values
        self.num_vehicles = num_vehicles
        
        if city_names is not None:
            self.city_names = city_names
            self.df = self.df[self.df['name'].isin(city_names)]
        else:
            self.city_names = self.all_city
    
    def create_data_model(self):
        data = {}
        temp = pd.read_excel('distance.xlsx', index_col=0)
        temp = temp[(temp.index.isin(self.city_names))][self.city_names]
#         print(temp)
    
        data['distance_matrix'] = temp.values / 1000
        data['num_vehicles'] = self.num_vehicles
        data['depot'] = 0
        return data
    
    
    def get_solution(self, manager, routing, solution):
        
        distance_list = []
        route_list = []
        
        for vehicle_id in range(self.num_vehicles):
            route_distance = 0
            route = []
            
            index =  routing.Start(vehicle_id)
            
            while not routing.IsEnd(index):
                #plan_output += ' {} ->'.format(city_names[manager.IndexToNode(index)])
                index_show = manager.IndexToNode(index)
                route.append(index_show)
                previous_index = index
                index = solution.Value(routing.NextVar(index))
                route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)

            route_list.append(route)
            distance_list.append(route_distance)
                
        return route_list, distance_list



    def add_distance_dimension(self,routing,transit_callback_index):
        # 添加距离约束
        dimension_name = 'Distance'
        routing.AddDimension(
            transit_callback_index,
            0,  # no slack
            10000,  # 车辆最大行驶距离
            True,  # start cumul to zero
            dimension_name)
        
        distance_dimension = routing.GetDimensionOrDie(dimension_name)
        # 尽量减少车辆之间的最大距离
        distance_dimension.SetGlobalSpanCostCoefficient(100)
 
    
    def work(self):
        data = self.create_data_model()
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
        routing = pywrapcp.RoutingModel(manager)
        
        def distance_callback(from_index, to_index):
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['distance_matrix'][from_node][to_node]
        
        
        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
        
        self.add_distance_dimension(routing,transit_callback_index)
        
        
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
        # 求解路径规划
        solution = routing.SolveWithParameters(search_parameters)
#         print('solution:', solution)
        
        route, route_distance = self.get_solution(manager, routing, solution)
        return route, route_distance

In [39]:
if __name__ == '__main__':
    model = tsp(num_vehicles = 4)
#     model.create_data_model()
    route_list, distance_list = model.work()
    print(route_list)
    print(distance_list)

[[0, 7, 18, 19, 17, 22, 6], [0, 21], [0, 5, 20, 23, 24, 26, 27, 25, 12, 32, 11, 31, 30, 13, 16], [0, 14, 28, 29, 10, 9, 8, 15, 1, 4, 2, 3]]
[6341, 7096, 6749, 6845]
